In [1]:
import os
if '_cwd_set' not in locals(): locals()['_cwd_set'] = os.chdir(os.path.dirname(os.getcwd()))
import omnifig as fig
fig.initialize()
from src.jimports import *
from src.util import set_default_device, repo_root, data_root
from src.dataset import RawCOCO, SimpleCOCO, RawCOCOCaptions, COCOCaptions, COCO, MNIST
device = set_default_device();

Selecting CUDA device 2 with 4026 MiB free memory and 100% utilization


In [2]:
cfg = fig.create_config(*'norm intv/module m/ced-man a/mlp d/coco-emb'.split())
# print(cfg)

In [2]:
loc = Path('/data/felix/cache/checkpoints/ae128_coco_20250114_230254/ckpt_020000')

In [3]:
cfg = fig.create_config(loc.joinpath('config.yaml'))

In [4]:
trainer = cfg.pull('trainer', silent=True)
model = trainer.model
modifier = trainer.intervention
traindataset = cfg.pull('dataset', silent=True)
valdataset = traindataset.as_eval()
traindataset

SimpleCOCO[106459](index, image, image_id, observation, instances, label)

In [5]:
traindataset.prepare(device=device);
valdataset.prepare(device=device)
system = Structured(traindataset, *trainer.gadgetry())
system.mechanize() # sync for gears and spaces
mech = system.mechanics()
model.prepare(device=device);
modifier.prepare(device=device);
for p in model.parameters():
    p.requires_grad = False
for p in modifier.parameters():
    p.requires_grad = False

In [6]:
model.load_checkpoint(path=loc.joinpath('model'));
modifier.load_checkpoint(path=loc.joinpath('interventions'));

/home/felix/workspace/clones/foundation/omnilearn/machines.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(path)
/home/felix/workspace/remote/src/inte

In [7]:
batch = traindataset.batch(24, model)
batch

Batch({size}, {num_iterations}, {drawn_batches}, {drawn_samples}, {index}, {epochs}, {initial_seed}, {epoch_seed}, {epoch_offset}, {image}, {image_id}, {observation}, {instances}, {label}, {latent}, {reconstruction}, {loss_rec}, {prediction})

In [8]:
rec = batch['reconstruction']
obs = batch['observation']
lbl = batch['label']
pred = batch['prediction']
rec.shape, pred.shape, lbl.shape

(torch.Size([24, 768]), torch.Size([24, 80]), torch.Size([24, 80]))

In [9]:
lbl[:5,:6]

tensor([[False, False, False, False, False, False],
        [ True, False, False, False, False, False],
        [False, False, False, False, False, False],
        [ True, False,  True, False, False,  True],
        [False, False, False, False, False, False]], device='cuda:2')

In [10]:
pred[:5,:6]

tensor([[ -6.6737,  -9.7150,  -7.5876, -11.0463, -12.7403,  -9.9729],
        [  8.4911,  -2.4404,  -0.5635,  -7.0519,  -8.1696,  -6.3583],
        [ -3.2721, -13.6740, -10.5507, -13.9637, -15.3645, -17.8923],
        [  5.4396,  -1.9623,  -0.1921,  -4.2699, -10.0154,  -3.4960],
        [ -4.2727,  -4.0172,  -5.3556, -11.6546, -12.4487, -12.7257]],
       device='cuda:2')

In [32]:
cond = modifier.get_conditioning(torch.as_tensor([1], device=device), torch.as_tensor([0], device=device))
cond = modifier.get_conditioning(torch.as_tensor([-1], device=device), torch.as_tensor([0], device=device))
cond.shape

torch.Size([1, 768])

In [26]:
latent = batch['latent']
latent.shape

torch.Size([24, 128])

In [33]:
intv = modifier.apply_intervention(latent[[1]], cond)
intv.shape

torch.Size([1, 128])

In [30]:
lbl[1]

tensor([ True, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,  True,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False, False, False,
         True, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False],
       device='cuda:3')

In [29]:
pred[1]

tensor([ 16.7824,  -6.8958,  -3.5942,  -7.1565, -15.4173,  -8.8879, -11.6496,
         -6.8098,  -9.0031,  -9.1677, -10.9302,  -6.3289,  -7.1757,  -3.1761,
         -8.6532,  -9.6531,  -7.4476, -12.0742, -13.9844, -15.1741, -14.5410,
        -11.7243, -13.9237, -15.0946,  -4.2062,  -6.0768,  -1.8894,  -5.9414,
         -9.4069, -13.8676, -12.4716, -14.5575, -10.0074, -18.5087, -17.6825,
         -9.9827, -15.9454, -13.7271, -14.9815,  -2.1873,  -2.6575,   0.1688,
         -0.1500,  -1.8403,  -3.3125,  -2.6015,  -6.3909,  -5.0885,  -0.4016,
         -6.0098,  -1.8677,  -2.4695,  -5.8093,  -3.1683,  -1.7097,  -5.9743,
          0.1463,  -4.0002,  -1.2560, -10.7558,   1.6960, -15.2440,  -2.8953,
         -5.4333,  -9.6435, -10.2609,  -7.2347,  -4.7824,  -6.6590,  -6.1437,
         -9.4887,  -6.8424,  -4.1618,  -5.2247,  -5.4061,  -3.1431,  -6.7401,
         -8.3932, -11.4496, -12.2338], device='cuda:3')

In [34]:
model.predict(intv)

tensor([[-1.0094e+00, -6.6570e+00, -2.6569e+00, -2.4035e+00, -8.4497e+00,
         -2.7963e+00, -1.0747e+01, -3.9825e+00, -9.5988e+00, -5.4661e+00,
         -1.0618e+01, -6.2455e+00, -6.3298e+00, -4.1514e+00, -7.4474e+00,
         -7.6641e+00, -8.4393e+00, -7.9666e+00, -1.2860e+01, -1.0908e+01,
         -1.3986e+01, -1.1588e+01, -8.3818e+00, -1.1853e+01, -4.5196e+00,
         -4.7303e+00, -2.0868e+00, -7.5000e+00, -5.7339e+00, -8.5720e+00,
         -1.1925e+01, -1.3579e+01, -1.0108e+01, -2.1130e+01, -1.2853e+01,
         -1.2159e+01, -1.6089e+01, -8.6460e+00, -1.5116e+01, -3.9588e+00,
         -5.4662e+00, -2.3741e-01, -3.0914e+00, -4.4981e+00, -3.3664e+00,
         -1.9679e+00, -9.7385e-01, -5.5971e+00, -1.5464e+00, -5.5428e+00,
         -2.8485e+00, -5.0852e+00, -6.9527e+00, -2.8262e+00, -5.3289e+00,
         -4.2072e+00, -4.7548e-01, -1.6194e+00, -2.1201e+00, -4.2561e+00,
          1.0847e-02, -1.0371e+01, -9.4079e-01, -3.1928e+00, -6.2424e+00,
         -6.7952e+00, -3.0962e+00, -3.

In [12]:
fullddata = COCO(eval_split=-0.1)
fullddata.prepare(device=device);
fullddata

COCO[59169](index, text_features, label, image_id, caption_id, caption, image, image_features)

In [17]:
# val_img_ids = valdataset.get_image_id(np.arange(valdataset.size))
# fiids = fullddata.get_image_id(np.arange(fullddata.size))
# fiid_map = {fiid: i for i, fiid in enumerate(fiids)}
# safe_ids = set(fiids.tolist()).intersection(set(val_img_ids.tolist()))
# inds = np.array([fiid_map[fiid] for fiid in safe_ids])
# inds.sort()
# save_json(inds.tolist(), 'save_inds.json')
inds = load_json('save_inds.json')
inds = np.array(inds)
len(inds), inds[:5]

(4135, array([116, 149, 183, 192, 235]))

In [57]:
batch = Context(fullddata, model, DictGadget({'index': inds, 'size': len(inds)}))
batch

Context({index}, {text_features}, {label}, {image_id}, {caption_id}, {caption}, {image}, {image_features}, {latent}, {reconstruction}, {loss_rec}, {prediction}, {size})

In [58]:
recs = batch['reconstruction']
recs.shape

GrabError: 'reconstruction' failed due to: 'latent' failed due to: 'observation' failed due to: missing gadget for 'observation'

In [6]:
raw = '''_base: [m/optim/intv, uni]
_meta: {script_name: train}
amsgrad: true
batch-size: 1024
budget: 100000
ckpt-freq: 10000
classifier: {_type: mlp, hidden: <>cls-hidden}
cls-hidden: [512, 512]
cls-wt: 1
dataset:
  _type: simple-coco
  app: {embedding: observation}
  split: train
dec-hidden: [512, 512]
decoder: {_type: mlp, hidden: <>dec-hidden}
device: cuda:0
enc-hidden: [512, 512]
encoder: {_type: mlp, hidden: <>enc-hidden}
env:
  criterion:
    _type: multi-loss
    wts: {cls: <>cls-wt, rec: <>rec-wt}
  detacher:
    _type: detacher
    app: {detached: latent_d, original: latent}
  gt-intervention: {_type: intervention/label}
  intv-criterion:
    _type: multi-loss
    app: {loss: loss_intv}
    wts: {mag: <>mag-wt, man: <>man-wt}
  manifold:
    _type: mechanism
    content: [<>env.task, <>model]
    external: {accuracy: accuracy_man, loss_cls: loss_man, prediction: prediction_man}
    internal: {label: intervention, latent: response}
  probe_response:
    _type: mechanism
    content: [<>model]
    external: {latent: response}
    internal: {latent: probe, observation: reconstruction}
  task:
    _type: multi-classification
    app: {loss: loss_cls, macro: accuracy}
  viz_intv_mag:
    _type: vecinfo
    app: {magnitude: intv_mag, scale: intv_scale, vector: probe}
  viz_latent_mag:
    _type: vecinfo
    app: {magnitude: latent_mag, scale: latent_scale, vector: latent}
  viz_pred_stats:
    _type: prediction-gap
    app: {ambient: prediction, intervened: prediction_man}
  viz_support:
    _type: support
    app: {src: latent, tgt: probe}
eval-batch-size: <>batch-size
eval-freq: 1000
eval-split: 0.1
events:
  checkpointer: {_type: checkpointer, freq: <>ckpt-freq}
  evaluator: {_type: evaluator, eval-reporter: <>events.monitor, freq: <>eval-freq,
    metrics: <>wandb}
  monitor: {_type: wandb, freqs: <>wandb, max-imgs: 12}
intervention:
  _type: intervention/module
  app: {ambient: latent_d}
  condition: /data/felix/cache/static/coco-bert-text-interventions.h5
  module: <>modifier
  normalize: true
intervention_optim: {_type: avoid-adam, objective: loss_intv}
latent-dim: 128
log: {accuracy: true, accuracy_man: true, loss_man: true, loss_rec: true}
lr: 0.001
mag-wt: 0
man-wt: 1
mod-hidden: [256]
model:
  _mod: {supervised: true}
  _type: norm-ae
  app: {features: latent, loss: loss_rec}
modifier: {_type: mlp, hidden: <>mod-hidden}
optimizer: {_type: adam, objective: loss}
planner: {_type: planner}
project-name: '{dataset.name}-ced'
rec-wt: 100
reporter: {_type: reporter}
saveroot: /data/felix/cache/checkpoints
trainer: {_type: intervention-trainer}
wandb: {accuracy: 10, accuracy_man: 10, amd: 100, changed_f1: 500, intv_comparison: 1000,
  intv_mag: 100, intv_scale: 100, latent_mag: 100, latent_scale: 100, loss_cls: 10,
  loss_mag: false, loss_man: 10, loss_rec: 10, proximity_stats: 500, rec_comparison: 1000,
  unchanged_f1: 500}
wandb-dir: /data/felix/cache/wandb'''
full = raw
# full = '\n'.join([l[4:] for l in raw.split('\n')])
Path('/data/felix/cache/checkpoints/ae128_coco_20250114_230254/ckpt_020000/config.yaml').write_text(full);